CS4001/4042 Assignment 1, Part B, Q3
---

Besides ensuring that your neural network performs well, it is important to be able to explain the model’s decision. **Captum** is a very handy library that helps you to do so for PyTorch models.

Many model explainability algorithms for deep learning models are available in Captum. These algorithms are often used to generate an attribution score for each feature. Features with larger scores are more ‘important’ and some algorithms also provide information about directionality (i.e. a feature with very negative attribution scores means the larger the value of that feature, the lower the value of the output).

In general, these algorithms can be grouped into two paradigms:
- **perturbation based approaches** (e.g. Feature Ablation)
- **gradient / backpropagation based approaches** (e.g. Saliency)

The former adopts a brute-force approach of removing / permuting features one by one and does not scale up well. The latter depends on gradients and they can be computed relatively quickly. But unlike how backpropagation computes gradients with respect to weights, gradients here are computed **with respect to the input**. This gives us a sense of how much a change in the input affects the model’s outputs.





---



---



In [ ]:
!pip install captum

In [ ]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

import torch
import torch.nn as nn

from captum.attr import Saliency, InputXGradient, IntegratedGradients, GradientShap, FeatureAblation

import matplotlib.pyplot as plt
from sklearn import preprocessing

from sklearn.metrics import mean_squared_error, r2_score


> First, load the dataset following the splits in Question B1. To keep things simple, we will **limit our analysis to numeric / continuous features only**. Drop all categorical features from the dataframes. Do not standardise the numerical features for now.



In [ ]:
# TODO: Enter your code here

df = pd.read_csv('hdb_price_prediction.csv')

numeric_df = df.drop(columns=['town', 'month', 'full_address', 'nearest_stn', 'storey_range', 'flat_model_type', ])

train_df = numeric_df[numeric_df['year'] <= 2019]
train_df = train_df.drop(columns=['year'])
val_df = numeric_df[numeric_df['year'] == 2020]
val_df = val_df.drop(columns=['year'])
test_df = numeric_df[numeric_df['year'] == 2021]
test_df = test_df.drop(columns=['year'])


> Follow this tutorial to generate the plot from various model explainability algorithms (https://captum.ai/tutorials/House_Prices_Regression_Interpret).
Specifically, make the following changes:
- Use a feedforward neural network with 3 hidden layers, each having 5 neurons. Train using Adam optimiser with learning rate of 0.001.
- Use Saliency, Input x Gradients, Integrated Gradients, GradientSHAP, Feature Ablation


In [ ]:
# TODO: Enter your code here

# Prepare data
X_train = torch.from_numpy(train_df.drop(columns=['resale_price']).to_numpy()).float() # torch.Size([64057, 6])
y_train = torch.tensor(train_df['resale_price'].values).view(-1, 1).float() # torch.Size([64057, 1])

X_test = torch.from_numpy(test_df.drop(columns=['resale_price']).to_numpy()).float() # torch.Size([29057, 6])
y_test = torch.tensor(test_df['resale_price'].values).view(-1, 1).float() # torch.Size([29057, 1])


datasets = torch.utils.data.TensorDataset(X_train, y_train)
train_iter = torch.utils.data.DataLoader(datasets, batch_size=10, shuffle=True)

# Set default hyperparams
batch_size = 50
num_epochs = 200
learning_rate = 0.001
size_hidden1 = 5
size_hidden2 = 5
size_hidden3 = 5
size_output = 1

# Build model
class myModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Only 6 input features after dropping categorical features and the target feature 'resale_price'
        self.lin1 = nn.Linear(6, size_hidden1)
        self.relu1 = nn.ReLU()
        self.lin2 = nn.Linear(size_hidden1, size_hidden2)
        self.relu2 = nn.ReLU()
        self.lin3 = nn.Linear(size_hidden2, size_hidden3)
        self.relu3 = nn.ReLU()
        self.lin4 = nn.Linear(size_hidden3, size_output)

    def forward(self, input):
        return self.lin4(self.relu3(self.lin3(self.relu2(self.lin2(self.relu1(self.lin1(input)))))))

# Train model
criterion = nn.MSELoss(reduction='sum')
def train(model_inp, train_iter, num_epochs):
    optimizer = torch.optim.Adam(model_inp.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0 # for each epoch
        for inputs, labels in train_iter:
            # forward pass
            outputs = model_inp(inputs)
            # defining loss
            loss = criterion(outputs, labels)
            # zero the parameter gradients
            optimizer.zero_grad()
            # computing gradients
            loss.backward()
            # accumulating running loss
            running_loss += loss.item()
            # updated weights based on computed gradients
            optimizer.step()
        if epoch % 20 == 0:    
            print('Epoch [%d]/[%d] running accumulative loss across all batches: %.3f' %
                  (epoch + 1, num_epochs, running_loss))
        running_loss = 0.0
    return model_inp



In [ ]:
model = myModel()
model.train()
model = train(model, train_iter, num_epochs)

# Save model
torch.save(model, 'unscaled_model')

In [ ]:
# Evaluate model
model.eval()
outputs = model(X_test)
err = np.sqrt(mean_squared_error(outputs.detach().numpy(), y_test.detach().numpy()))

print('model rsme: ', err)

In [ ]:
# Attributions
def saliency(model, test):
	sal = Saliency(model)
	sal_attr_test = sal.attribute(test)
	return sal_attr_test

def	input_x_gradient(model, test):
	ixg = InputXGradient(model)
	ixg_attr_test = ixg.attribute(test)
	return ixg_attr_test

def integrated_gradients(model, test):
	ig = IntegratedGradients(model)
	ig_attr_test = ig.attribute(test, n_steps=50)
	return ig_attr_test

def gradient_shap(model, test, train):
	gs = GradientShap(model)
	# use the entire training dataset as the distribution of baselines
	gs_attr_test = gs.attribute(test, train)
	return gs_attr_test

def feature_ablation(model, test):
	fa = FeatureAblation(model)
	fa_attr_test = fa.attribute(test)
	return fa_attr_test

def attr_test_norm_sum(attr_test):
	attr_test_sum = attr_test.detach().numpy().sum(0)
	attr_test_norm_sum = attr_test_sum / np.linalg.norm(attr_test_sum, ord=1)
	return attr_test_norm_sum

def get_y_axis_lin_weight(model):
	lin_weight = model.lin1.weight[0].detach().numpy()
	y_axis_lin_weight = lin_weight / np.linalg.norm(lin_weight, ord=1)
	return y_axis_lin_weight

# Plottings
def plot(x_test, attr_test_norm_sum, y_axis_lin_weight, title='Comparing input feature importances across multiple algorithms and learned weights', legends=['Saliency', 'Input X Gradients', 'Integrated Gradients', 'GradientSHAP', 'Feature Ablation', 'Weights']):
	width = 0.14
	plt.figure(figsize=(20, 10))
	ax = plt.subplot()
	ax.set_title(title)
	ax.set_ylabel('Attributions')
	FONT_SIZE = 16
	plt.rc('font', size=FONT_SIZE)            # fontsize of the text sizes
	plt.rc('axes', titlesize=FONT_SIZE)       # fontsize of the axes title
	plt.rc('axes', labelsize=FONT_SIZE)       # fontsize of the x and y labels
	plt.rc('legend', fontsize=FONT_SIZE - 4)  # fontsize of the legend

	x_axis_data = np.arange(x_test.shape[1])
	feature_names = list(train_df.drop(columns=['resale_price']).columns)
	x_axis_data_labels = list(map(lambda idx: feature_names[idx], x_axis_data))

	i = 1
	colours = ['#A90000', '#34b8e0', '#eb5e7c', '#4260f5', '#49ba81', 'grey']
	while i <= len(attr_test_norm_sum):
		ax.bar(x_axis_data + i * width, attr_test_norm_sum[i-1], width, align='center', alpha=0.75, color=colours[i-1])
		i += 1
	ax.bar(x_axis_data + i * width, y_axis_lin_weight, width, align='center', alpha=1.0, color='grey')

	ax.autoscale_view()
	plt.tight_layout()
	ax.set_xticks(x_axis_data + 0.5)
	ax.set_xticklabels(x_axis_data_labels)
	ax.grid(axis='y')
	plt.legend(legends, loc=3)
	plt.show()

In [ ]:
# For the attribution calculations, we will limit the test input size to 5000

# X_test_5000 = torch.empty((5000, 6), dtype=torch.float32)
# for idx in range(5000):
# 	i = random.randint(0, X_test.shape[0]-1)
# 	X_test_5000[idx] = X_test[i]

X_test_5000 = X_test[:1000]

sal_attr_test = saliency(model, X_test_5000)
ixg_attr_test = input_x_gradient(model, X_test_5000)
ig_attr_test = integrated_gradients(model, X_test_5000)
gs_attr_test = gradient_shap(model, X_test_5000, X_train)
fa_attr_test = feature_ablation(model, X_test_5000)

attr_test_norm_sum_list = []
for attr_test in [sal_attr_test, ixg_attr_test, ig_attr_test, gs_attr_test, fa_attr_test]:
	attr_test_norm_sum_list.append(attr_test_norm_sum(attr_test))

y_axis_lin_weight = get_y_axis_lin_weight(model)

plot(X_test_5000, attr_test_norm_sum_list, y_axis_lin_weight)


> Train a separate model with the same configuration but now standardise the features via **StandardScaler** (fit to training set, then transform all). State your observations with respect to GradientShap and explain why it has occurred.
(Hint: Many gradient-based approaches depend on a baseline, which is an important choice to be made. Check the default baseline settings carefully.)


In [ ]:
y_train_df = train_df['resale_price'] # (64057,)
torch.tensor(y_train_df.values).shape

In [ ]:
# TODO: Enter your code here

# Scale data
X_train_df = train_df.drop(columns=['resale_price']) # (64057, 6) 
X_test_df = test_df.drop(columns=['resale_price']) # (29057, 6)
y_train_df = train_df['resale_price'] # (64057,)
y_test_df = test_df['resale_price'] # (29057,)

x_standard_scaler = preprocessing.StandardScaler()
# y_standard_scaler = preprocessing.StandardScaler()
x_standard_scaler.fit(X_train_df)
# y_standard_scaler.fit(torch.tensor(y_train_df.values).view(-1, 1))
# torch.tensor(y_train_df.values).view(-1, 1).shape -> torch.Size([64057, 1])

x_train_scaled = x_standard_scaler.transform(X_train_df)
X_train_scaled_tensor = torch.tensor(x_train_scaled, dtype=torch.float32) #torch.Size([64057, 6]) 
# y_train_scaled = y_standard_scaler.transform(y_train_df.values.reshape(-1, 1))
# y_train_scaled_tensor = torch.tensor(y_train_scaled).float() #torch.Size([64057, 1])
x_test_scaled = x_standard_scaler.transform(X_test_df)
X_test_scaled_tensor = torch.tensor(x_test_scaled, dtype=torch.float32) #torch.Size([29057, 6]) 
# y_test_scaled = y_standard_scaler.transform(y_test_df.values.reshape(-1, 1))
# y_test_scaled_tensor = torch.tensor(y_test_scaled).float() #torch.Size([29057, 1])

y_train_tensor = torch.tensor(y_train_df.values, dtype=torch.float32) #torch.Size([64057])
y_test_tensor = torch.tensor(y_test_df.values, dtype=torch.float32) #torch.Size([29057])


# Load data
datasets_scaled = torch.utils.data.TensorDataset(X_train_scaled_tensor, y_train_tensor)
# datasets_scaled = torch.utils.data.TensorDataset(X_train_scaled_tensor, y_train_scaled_tensor)
train_iter_scaled = torch.utils.data.DataLoader(datasets_scaled, batch_size=10, shuffle=True)



In [ ]:
# Create and train model
model_scaled = myModel()
model_scaled.train()
model_scaled = train(model_scaled, train_iter_scaled, num_epochs)

# Evaluate model
model_scaled.eval()
outputs = model_scaled(X_test_scaled_tensor)
err = np.sqrt(mean_squared_error(outputs.detach().numpy(), y_test_tensor.detach().numpy()))
# err = np.sqrt(mean_squared_error(outputs.detach().numpy(), y_test_scaled_tensor.detach().numpy()))

print('model err: ', err)

In [ ]:

# Save model
torch.save(model_scaled, 'scaled_model')

In [ ]:
# X_test_5000_scaled = torch.empty((5000, 6), dtype=torch.float32)
# for idx in range(5000):
# 	i = random.randint(0, X_test_scaled_tensor.shape[0]-1)
# 	X_test_5000_scaled[idx] = X_test_scaled_tensor[i]

X_test_5000_scaled = X_test_scaled_tensor[:1000]

sal_scaled = saliency(model_scaled, X_test_5000_scaled)
ixg_scaled = input_x_gradient(model_scaled, X_test_5000_scaled)
ig_scaled = integrated_gradients(model_scaled, X_test_5000_scaled)
gradshap_scaled = gradient_shap(model_scaled, X_test_5000_scaled, X_train_scaled_tensor)
fa_scaled = feature_ablation(model_scaled, X_test_5000_scaled)

attr_test_norm_sum_list_scaled = []
for attr_test in [sal_scaled, ixg_scaled, ig_scaled, gradshap_scaled, fa_scaled]:
	attr_test_norm_sum_list_scaled.append(attr_test_norm_sum(attr_test))

y_axis_lin_weight_scaled = get_y_axis_lin_weight(model_scaled)

plot(X_test_5000_scaled, attr_test_norm_sum_list_scaled, y_axis_lin_weight_scaled)

In [ ]:
table = pd.DataFrame({'Scaled': [128, 256, 512, 1024],
                   'Non-scaled': [training_times[128], training_times[256], training_times[512], training_times[1024]]
                  })

table

Table of GradientSHAP attributions
|   | Non-scaled | Scaled |
| --- | --- | --- |
| dist_to_nearest_stn | -0.17 | -0.12 |
| dist_to_dhoby | 0.62 | 0.03 |
| degree_centrality | 0 | 0.06
| eigenvector_centrality | 0 | 0
| remaining_lease_years | -0.08 | 0.65
| floor_area_sqm | -0.14 | -0.1

In the model trained and tested on <ins>scaled</ins> data, GradientSHAP indicates that <ins>*remaining_lease_years*</ins> is the <ins>most important feature</ins> among the 6 continuous features, scoring an attribution of nearly 0.65. However, in the model trained on <ins>un-scaled</ins> data, GradientSHAP indicates that <ins>*dist-to_dhoby*</ins> is the <ins>most important</ins> feature, scoring approximately 0.62, whereas *remaining_lease_years* only scored approximately -0.08. <br>
Additionally, for the feature <ins>*remaining_lease_years*</ins>, in the model trained and tested on <ins>scaled</ins> data, it shows a <ins>positive attribution of approximately 0.65</ins>, but for the <ins>un-scaled</ins> case, it showed a <ins>negative attribution of approximately -0.08</ins>
<br> <br>
This is because GradientSHAP computations are based on the baseline, which is X_train and X_train_scaled in our two models respectively. When the data is scaled, it reduces the impact of the outlier data points on the model trained and hence results in different GradientSHAP attributions in the two models.

Read https://distill.pub/2020/attribution-baselines/ to build up your understanding of Integrated Gradients (IG). Reading the sections before the section on ‘Game Theory and Missingness’ will be sufficient. Keep in mind that this article mainly focuses on classification problems. You might find the following [descriptions](https://captum.ai/docs/attribution_algorithms) and [comparisons](https://captum.ai/docs/algorithms_comparison_matrix) in Captum useful as well.


Then, answer the following questions in the context of our dataset:

> Why did Saliency produce scores similar to IG?


\# TODO: \<Enter your answer here\>

In both the scaled and non-scaled cases, Saliency did not produce scores similar to IG, especially in the non-scaled case where the score for Saliency is positive while the score for IG is negative.
<br>
This is because IG score is obtained after a multiplication with the input, which is absent in Saliency. Hence, depening on the input value, the IG score can be very different from the Saliency score. 
Additionally, IG also takes into account alpha. It considers the integral of (alpha multipied by the gradient of the output with respect to the input) over alpha, while Saliency only takes the gradient of output with respect to the input.

> Why did Input x Gradients give the same attribution scores as IG?


\# TODO: \<Enter your answer here\>

Input x Gradients give attribution scores that are almost completely identical as IG (only extremely small differences can be spotted for features *dist_to_dhoby* and *floor_area_sqm). <br>
This is because in our context here, both Input x Gradients and IG use the zero-vector as their baselines. <br>
Hence, the computation for IG reduces to the product of the input and the integral of (alpha multipied by the gradient of the output with respect to the input) over alpha.
Thus, Input x Gradients and IG give almost identical attribution scores